<center style = "font-family: 'Lucida Console', 'Courier New', monospace;">
    <img src = "https://takelessons.com/blog/wp-content/uploads/2014/05/Essay.jpg" width=400 height = 200>
    <h1 style = "background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%);border-radius: 20px; font-size:30px">Feedback Prize - Evaluating Student Writing 🧑‍✏️🧾</h1>
</center>

<div style = "background: rgb(224,224,224);border-radius: 42px;">
    <h1 style = "font-family: Consolas; text-align:center; color:#FF69B4">Introduction</h1>
    <h2 style = "font-family: Consolas; text-align:center">Why this Competition ❓</h2>
    <p style = "font-family : Lucida Sans Typewriter">
     One way to help students improve their writing is via automated feedback tools, which evaluate student writing and provide personalized feedback. There are currently numerous automated writing feedback tools, but they all have limitations. Many often fail to identify writing structures, such as thesis statements and support for claims, in essays or do not do so thoroughly. 
    </p>
    <h2 style = "font-family: Consolas; text-align:center">Goal of Competition 🥅</h2>
    <p style = "font-family : Lucida Sans Typewriter">
    Our goal is to identify elements in student writing. More specifically, you will automatically segment texts and classify argumentative and rhetorical elements in essays written by 6th-12th grade students. You'll have access to the largest dataset of student writing ever released in order to test your skills in natural language processing, a fast-growing area of data science.
    </p>
</div>

<h2 style = "font-family: Consolas">More Details</h2>
<p style = "font-family : Lucida Sans Typewriter">Check <a href = "https://www.kaggle.com/c/feedback-prize-2021">competition page</a> for details</p>
<h2 style = "font-family : Comic Sans MS">Let's dive in ⬇️</h2>

<center><img src = "https://img.shields.io/badge/Upvote-If%20you%20found%20this%20notebook%20useful-blue" width=400 height = 400></center>

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Import Libraries 📚</h1>

In [ ]:
import os

import pandas as pd
from pandas_profiling import ProfileReport

import spacy
import nltk
import numpy as np
import ast

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import wordcloud
from colorama import Fore, Back, Style

import wandb

from IPython.display import IFrame
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; ">Initialize Constants 🔰</h1>

In [ ]:
train_df = pd.read_csv("../input/feedback-prize-2021/train.csv")
num_rows = len(train_df)
TRAIN_PATH = "../input/feedback-prize-2021/train"

train_files = os.listdir(TRAIN_PATH)
train_ids = [i.split(".")[0] for i in train_files]
train_paths = [os.path.join(TRAIN_PATH,f) for f in train_files]

classes = ["Claim", "Evidence", "Position", "Concluding Statement", "Lead", "Counterclaim", "Rebuttal", "None"]
crma = {
    classes[0] : Fore.BLUE,
    classes[1] : Fore.RED,
    classes[2] : Fore.MAGENTA,
    classes[3] : Fore.YELLOW,
    classes[4] : Fore.GREEN,
    classes[5] : Fore.WHITE,
    classes[6] : Fore.CYAN,
    classes[7] : Fore.LIGHTBLUE_EX
}
sr_crma = Style.RESET_ALL

train_df.head(2)

<h1 style = "font-family: 'Lucida Console', 'Courier New', monospace; background: rgb(44,169,201);
background: linear-gradient(180deg, rgba(44,169,201,1) 0%, rgba(1,94,125,1) 100%); border-radius: 20px; font-size:30px; text-align:center; "> Data Preprocessing ⚒️🔬</h1>

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Necessary Functions</h2>

In [ ]:
def read_txt(filepath):
    with open(filepath, "r") as f:
        text = f.read()
    return text

def get_row(text, df, pos, count):
    row_curr, row_next = df.iloc[pos], df.iloc[pos+1]
    txt_id, discourse_id = row_curr.id, row_curr.discourse_id
    
    if pos == 0 and row_curr.discourse_start!=0:
        start, end = 0, int(row_curr.discourse_start)
        start_word, end_word = 0, int(row_curr.predictionstring.split(" ")[-1])
    elif pos == len(df)-1 and row_curr.discourse_end!=len(text):
        start, end = int(row_curr.discourse_end), len(text)
        start_word, end_word = int(row_curr.predictionstring.split(" ")[-1])+1, len(text.split(" "))-1
    else:
        start, end = int(row_curr.discourse_end), int(row_next.discourse_start)
        start_word = int(row_curr.predictionstring.split(" ")[-1])+1
        end_word = int(row_next.predictionstring.split(" ")[0])
    
    pred_str = [str(start_word+i) for i in range(end_word-start_word)] 
    pred_str = " ".join(pred_str)
    discourse_text = text[start:end]
    return [
            txt_id,discourse_id,
            start, end,
            discourse_text,
            "None", "None " + str(count),
            pred_str
           ]

In [ ]:
##-----------------------------------------------
#Getting list of text that is missing in csv file
##-----------------------------------------------
none_ls = []
for txt_id in tqdm(train_ids):
    df = train_df[train_df["id"] == txt_id]
    text = read_txt(os.path.join(TRAIN_PATH,txt_id+".txt"))
    count = 0
    
    for i in range(len(df)-1):
        row_i0, row_i1 = df.iloc[i], df.iloc[i+1]
        if row_i1.discourse_start > row_i0.discourse_end+1:
            none_ls.append(get_row(text, df, i, count))
            count += 1
            
        if row_i1.discourse_end < len(text) and i == len(df)-1:
            start, end = int(row_i1.predictionstring.split(" ")[-1])+1, len(text.split(" "))
            pred_str = [str(start+i) for i in range(end-start)] 
            pred_str = " ".join(pred_str)
            none_ls.append([
                row_i1.id, row_i1.discourse_id,
                row_i1.discourse_end, len(text),
                text[int(row_i1.discourse_end):len(text)],
                "None", "None "+str(count),
                pred_str
            ])

In [ ]:
##---------------------------------------
#Adding text with None class to DataFrame
##---------------------------------------
none_df = pd.DataFrame(none_ls,columns=list(train_df.columns))
none_df.head()

df = pd.concat([train_df,none_df])
df = df.sort_values(by=["id","discourse_start"],ascending=[True,True]).reset_index(drop=True)
df.tail(3)

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Weights and Biases</h2>
    <center><img src = "https://i.imgur.com/KISYcqD.png" width=200 height = 200></center>
    <a href = "https://wandb.ai/anony-moose-172647/Feedback%20Prize%20-%20Evaluating%20Student%20Writing?workspace=user-shanmukh"; style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; text-align:center; font-size:25px">Checkout Dashboard created for this notebook</a>
    <p style = "font-family : Lucida Sans Typewriter">
    Weights and Biases is a set of Machine Learning tools used for experiment tracking, dataset versioning, and collaborating on ML projects. Weights and Biases is useful in many applications such as
    </p>  
    <ul>
          <li>Experiment Tracking</li>
          <li>Hyperparameter Tuning</li>
          <li>Data Visualization</li>
          <li>Data and model Versioning</li>
          <li>Collaborative Reports</li>
    </ul>
    <a href = "https://wandb.ai/site">Go to offocial website for more tutorials and Documentation</a>
</div>

In [ ]:
try:
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('''If you want to use your W&B account, Follow these steps :
            -> go to Add-ons {Below name of notebook} -> Secrets -> Add a new Secret
            -> Label = wandb_api
            -> Value = W&B access token from https://wandb.ai/authorize 
         ''')

In [ ]:
WANDB_CONFIG = {
   "project_name" : "Feedback Prize - Evaluating Student Writing",
   "group_name" : "PyTorch Training",
   "job_type_data" : "Data Visualization",
   "anonymity" : "must",
    "artifact" : "training_data"
}

run = wandb.init(
    project = WANDB_CONFIG["project_name"],
    group = WANDB_CONFIG["group_name"],
    job_type = WANDB_CONFIG["job_type_data"],
    anonymous= WANDB_CONFIG["anonymity"]
)

In [ ]:
wb_table = wandb.Table(columns = [
    'id', 'discourse_id', 'discourse_start', 'discourse_end',
    'discourse_text', 'discourse_type', 'discourse_type_num', 'predictionstring'
])

for i in range(len(df)):
    row = df.loc[i]
    wb_table.add_data(
        row["id"],
        row["discourse_id"],
        row["discourse_start"],
        row["discourse_end"],
        row["discourse_text"],
        row["discourse_type"],
        row["discourse_type_num"],
        row["predictionstring"]
    )
    
wandb.log({'Data Visualization': wb_table})
run.finish()

In [ ]:
frame = IFrame(run.url, width=1280, height=720)
frame

<div style = "font-family : Lucida Sans Typewriter;background: rgb(224,224,224);border-radius: 25px;">
    <h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Pandas Profiling</h2>
    <center><img src = "https://pandas-profiling.github.io/pandas-profiling/docs/assets/logo_header.png" width=200 height = 200></center>
    <p style = "font-family : Lucida Sans Typewriter">
    Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. Besides, if this is not enough to convince us to use this tool, it also generates interactive reports in web format that can be presented to any person, even if they don’t know programming.
    </p>  
    <a href = "https://pandas-profiling.github.io/pandas-profiling/">Go to offocial website for documentation</a>
</div>

In [ ]:
train_report = ProfileReport(df,title="Metadata of Training images")
train_report.to_file("./train_metadata.html")
train_report

In [ ]:
##------------------------------------
#Print a discourse coloured by classes
##------------------------------------
for cls in classes:
    print(f"{crma[cls]} {cls} {sr_crma} ", end=" ")

print("\n\n")
tmp_df = df[df["id"] == train_ids[7]]
for i in range(len(tmp_df)):
    row = tmp_df.iloc[i]
    print(f"{crma[row.discourse_type]}{row.discourse_text}{sr_crma}", end=" ")

In [ ]:
##----------------------
#Distribution of classes
##----------------------

classes_count = [list(df["discourse_type"]).count(i) for i in classes]

fig = px.bar(x = classes, y = classes_count, color = classes,color_continuous_scale="Emrld") 
fig.update_xaxes(title="Classes")
fig.update_yaxes(title = "Count")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Discourse Class Distribution ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

In [ ]:
##-----------------------
#Length of Text and Words
##-----------------------
text_length, word_length = {}, {}

for i,path in tqdm(enumerate(train_paths)):
    txt = read_txt(path)
    text_length[train_ids[i]] = len(txt)
    word_length[train_ids[i]] = len(txt.split(" "))
    
df["text_length"] = df["id"].map(lambda x : text_length[x])
df["word_length"] = df["id"].map(lambda x : word_length[x])

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Histogram(x=list(text_length.values()), name="Length of text"),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=list(word_length.values()), name="Number of words"),
    row=1, col=2
)

fig.update_layout(height=500, width=900, title_text="Histogram of length of text and number of words")
fig.show()

In [ ]:
##-------------------------------------------------
#Distribution of each class in different discourses
##-------------------------------------------------
count_dict = {
    classes[0] : [],
    classes[1] : [],
    classes[2] : [],
    classes[3] : [],
    classes[4] : [],
    classes[5] : [],
    classes[6] : [],
    classes[7] : []
}

for txt_id in tqdm(train_ids):
    tmp = df[df["id"] == txt_id]
    for c in classes:
        count_dict[c].append(len(tmp[tmp["discourse_type"] == c]))

In [ ]:
fig = make_subplots(rows=4, cols=2, start_cell="bottom-left")
r = [1,1,2,2,3,3,4,4]
c = [1,2,1,2,1,2,1,2]

for i,cls in enumerate(classes):
    fig.add_trace(go.Histogram(x=count_dict[cls], name=cls),row=r[i], col=c[i])

fig.update_layout(height=800, width=900, title_text="Distribution of each class in different discourses")
fig.show()

In [ ]:
##--------------------------------
#Information of each class in text
##--------------------------------
start, end, length, words = {}, {}, {}, {}

for cls in classes:
    tmp = df[df["discourse_type"] == cls]
    
    tmp["count"] = tmp["predictionstring"].map(lambda x : len(x.split(" ")))
    start[cls] = np.array(tmp["discourse_start"])/np.array(tmp["text_length"])
    end[cls] = np.array(tmp["discourse_end"])/np.array(tmp["text_length"])
    length[cls] = np.array(tmp["discourse_end"] - tmp["discourse_start"] + 1)/np.array(tmp["text_length"])
    words[cls] = np.array(tmp["count"])/np.array(tmp["text_length"])

In [ ]:
##---------------------------
#Start position of each Class
##---------------------------

fig = make_subplots(rows=4, cols=2)

for i,cls in enumerate(classes):
    fig.add_trace(go.Box(x=start[cls], name=cls),row=r[i], col=c[i])
    
fig.update_layout(height=800, width=900, title_text="Start Position of text for each class [Normalized]")
fig.show()

In [ ]:
##-------------------------
#End position of each Class
##-------------------------

fig = make_subplots(rows=4, cols=2)

for i,cls in enumerate(classes):
    fig.add_trace(go.Box(x=end[cls], name=cls),row=r[i], col=c[i])
    
fig.update_layout(height=800, width=900, title_text="End Position of text for each class [Normalized]")
fig.show()

In [ ]:
##----------------------------
#Length of text per each Class
##----------------------------

fig = make_subplots(rows=4, cols=2)

for i,cls in enumerate(classes):
    fig.add_trace(go.Box(x=length[cls], name=cls),row=r[i], col=c[i])
    
fig.update_layout(height=800, width=900, title_text="Length of text for each class [Normalized]")
fig.show()

In [ ]:
##-------------------------------------
#Number of words in text per each Class
##-------------------------------------

fig = make_subplots(rows=4, cols=2)

for i,cls in enumerate(classes):
    fig.add_trace(go.Box(x=words[cls], name=cls),row=r[i], col=c[i])
    
fig.update_layout(height=800, width=900, title_text="Number of words text for each class [Normalized]")
fig.show()

<h2 style = "font-family: 'Lucida Console', 'Courier New', monospace; border-radius: 20px; font-size:30px; text-align:center; ; color:#FF69B4">Word Cloud</h2>

In [ ]:
wcloud = wordcloud.WordCloud(stopwords=wordcloud.STOPWORDS, max_font_size=80, max_words=6000,
                      width = 600, height = 400,
                      background_color='black').generate(' '.join(txt for txt in df["discourse_text"]))

fig, ax = plt.subplots(figsize=(14,10))
ax.imshow(wcloud, interpolation='bilinear')
ax.set_axis_off()
plt.imshow(wcloud)

<br>
<h3 style = "font-family: Consolas; text-align:center; color:#FF0000">If you come this far, you could've got some insights from this notebook. An upvote would be very helpful :). Kindly comment if there are any doubts or mistakes</h3>

<center><img src = "https://img.shields.io/badge/Completed-The%20End-brightgreen" width=300 height = 300></center>